In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
import time
import random

In [1]:
def links(s,url):
    r= requests.get(url)
    #print (r)
    soup = BeautifulSoup(r.text, "html.parser")
    #print (soup)
    #self: Tag, key: str, default: str |
    l=[url]
    np=soup.find("a", class_="_1LKTO3")
    #print(np)
    new_links=[]
    c=0
    while True:
        if c==s:
             break
        r = requests.get(url)
        soup=BeautifulSoup(r.text, "lxml")
        np = soup.findAll("a", class_="_1LKTO3")
        if np[-1].text.strip()!='Next':
                next=i['href']
                break
        for i in np:
            if i.text.strip()=='Next':
                next=i['href']
                break
        cnp = "https://www.flipkart.com"+next
        print(cnp)
        l.append(cnp)
        url = cnp
        finder=soup.findAll('a',{'class':'_3bPFwb'})
        c=c+1
        for link in finder:
             new_links.append("https://www.flipkart.com"+link['href'])

    return new_links

In [11]:
def scrape_product_data(z,url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    df=pd.DataFrame({})
    time.sleep(1)
    response = requests.get(url, headers=headers)
    #print(response)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extracting title
        title = soup.find("span", {"class": "B_NuCI"})
        if title:
            title=title.text.strip()
        # Extracting price
        price = soup.find("div", {"class": "_30jeq3"})
        if price:
            price=price.text.strip()
        # Extracting ratings
        orgprice = soup.find("div", {"class": "_3I9_wc _2p6lqe"})
        if orgprice:
            orgprice=orgprice.text.strip()
        ratings = soup.find("div", {"class": "_3LWZlK _3uSWvT"})
        if ratings:
            ratings=ratings.text.strip()
        # Extracting reviews
        reviews_sum = soup.find("span", {"class": "_2_R_DZ"})
        if reviews_sum:
            reviews_sum=reviews_sum.text.strip() 
        seller=soup.find('span',{'class':'G6XhRU'})
        if seller:
            seller=seller.text.strip()
            
        reviews=soup.find("div", {'class':"col JOpGWq _33R3aa"})
        all_reviews=None
        
        if reviews:
            reviews=reviews.findAll('a')
            if reviews:
                reviews=reviews[-1]
                all_reviews='https://www.flipkart.com'+reviews['href']

        star=[0,0,0,0,0]
        if all_reviews:
            #print(all_reviews)
            r= requests.get(all_reviews)
            #print(r)
            soup1 = BeautifulSoup(r.text, "html.parser")
            #print(soup)
            stars=soup1.findAll('div',{'class':'_1uJVNT'})
            #print(stars)
            
            for i in range(len(stars)):
                star[i]=stars[4-i]
                if star[i]:
                    star[i]=star[i].text.strip()
                else:
                    star[i]=0
        data = {
            'code':[generate_unique_code()],
            'Title': [title],
            'Price': [price],
            'original price':[orgprice],
            'Rating': [ratings],
            'Url2':[all_reviews],
            'Reviews': [reviews_sum],
            'Url':[url],
            'Stars':[star],
            'Seller':[seller]
        }
        sep={}
        dark=soup.findAll('div',{'class':'col col-3-12 _2H87wv'})
        light=soup.findAll('div',{'class':'col col-9-12 _2vZqPX'})
        print(dark,light)
        for i in range(len(light)):
            sep[dark[i].text.strip()]=light[i].text.strip()
            #print(i.text)
        data.update(sep)
        
        df=pd.DataFrame(data)
        df['Total details']=[sep]

        print(z,url)
    else:
        print("Failed",url)

    return df
def generate_unique_code():
    characters = string.ascii_letters + string.digits  # All possible characters
    unique_code = ''.join(random.sample(characters, 10))  # Randomly sample 10 characters without replacement
    return unique_code

In [14]:
url = "\
https://www.flipkart.com/search?q=SASSAFRAS&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
l=links(25,url)
i=0
df=pd.DataFrame({})
for link in l:
    try:
        i=i+1
        df=pd.concat([df,scrape_product_data(i,link)])
        #print(i)
    except:
        print('failed',i,link)
        

https://www.flipkart.com/search?q=SASSAFRAS&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=2


In [1]:
#df.to_json('sassafrass.json')

In [10]:
url='https://www.flipkart.com/sassafras-women-bodycon-beige-dress/p/itmc6c13e50c3751?pid=DREGWMEGRHXHHXR5&lid=LSTDREGWMEGRHXHHXR5NRMQ16&marketplace=FLIPKART&q=SASSAFRAS&store=search.flipkart.com&srno=s_1_3&otracker=search&otracker1=search&fm=organic&iid=04be320f-9de6-41ed-beb7-77ae990bb30f.DREGWMEGRHXHHXR5.SEARCH&ppt=None&ppn=None&ssid=3a2weofdc00000001708279636087&qH=911e6d426b9cde98'
l=scrape_product_data(1,url)

[<div class="col col-3-12 _2H87wv">Color</div>, <div class="col col-3-12 _2H87wv">Length</div>, <div class="col col-3-12 _2H87wv">Fabric</div>, <div class="col col-3-12 _2H87wv">Pattern</div>, <div class="col col-3-12 _2H87wv">Ideal For</div>, <div class="col col-3-12 _2H87wv">Type</div>, <div class="col col-3-12 _2H87wv">Style Code</div>, <div class="col col-3-12 _2H87wv">Suitable For</div>, <div class="col col-3-12 _2H87wv">Sleeve Length</div>, <div class="col col-3-12 _2H87wv">Pack of</div>, <div class="col col-3-12 _2H87wv">Neck</div>, <div class="col col-3-12 _2H87wv">Fabric Care</div>, <div class="col col-3-12 _2H87wv">Sleeve</div>, <div class="col col-3-12 _2H87wv">Net Quantity</div>] [<div class="col col-9-12 _2vZqPX">Beige</div>, <div class="col col-9-12 _2vZqPX">Midi/Calf Length</div>, <div class="col col-9-12 _2vZqPX">Cotton Blend</div>, <div class="col col-9-12 _2vZqPX">Solid</div>, <div class="col col-9-12 _2vZqPX">Women</div>, <div class="col col-9-12 _2vZqPX">Bodycon</di

In [5]:
ur1='https://www.flipkart.com/sassafras-women-a-line-multicolor-dress/p/itmf850af7a4d4f4?pid=DREGPZGGW7XDY2JH&lid=LSTDREGPZGGW7XDY2JHRJHKC1&marketplace=FLIPKART&q=SASSAFRAS&store=search.flipkart.com&srno=s_1_15&otracker=search&otracker1=search&fm=organic&iid=2714fef2-8430-4b78-97f7-58a2da512cc5.DREGPZGGW7XDY2JH.SEARCH&ppt=None&ppn=None&ssid=4f5vq503ds0000001710020515588&qH=911e6d426b9cde98'
d=scrape_product_data(1,ur1)

TypeError: scrape_product_data() takes 1 positional argument but 2 were given

In [13]:
d

,code,Title,Price,original price,Rating,Url2,Reviews,Url,Stars,Seller,...,Ideal For,Type,Style Code,Suitable For,Sleeve Length,Pack of,Neck,Fabric Care,Net Quantity,Total details
0,butPsE7yWq,Women A-line Multicolor Dress,₹689,"₹2,299",4.1,https://www.flipkart.com/sassafras-women-a-lin...,90 ratings and 6 reviews,https://www.flipkart.com/sassafras-women-a-lin...,"[9, 3, 9, 16, 53]",SASSAFRAS,...,Women,A-line,SFDRSS10310-Beige,Western Wear,Half Sleeve,1,V-Neck,Dry Clean,1,"{'Color': 'Multicolor', 'Length': 'Above Knee/..."


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import string

def scrape_product_data(z,url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    time.sleep(1)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        title = soup.find("span", {"class": "B_NuCI"}).text.strip() if soup.find("span", {"class": "B_NuCI"}) else None
        price = soup.find("div", {"class": "_30jeq3"}).text.strip() if soup.find("div", {"class": "_30jeq3"}) else None
        orgprice = soup.find("div", {"class": "_3I9_wc _2p6lqe"}).text.strip() if soup.find("div", {"class": "_3I9_wc _2p6lqe"}) else None
        ratings = soup.find("div", {"class": "_3LWZlK _3uSWvT"}).text.strip() if soup.find("div", {"class": "_3LWZlK _3uSWvT"}) else None
        reviews_sum = soup.find("span", {"class": "_2_R_DZ"}).text.strip() if soup.find("span", {"class": "_2_R_DZ"}) else None
        seller = soup.find('span',{'class':'G6XhRU'}).text.strip() if soup.find('span',{'class':'G6XhRU'}) else None
        
        reviews = soup.find("div", {'class':"col JOpGWq _33R3aa"})
        all_reviews = None
        if reviews:
            reviews = reviews.findAll('a')
            if reviews:
                reviews = reviews[-1]
                all_reviews = 'https://www.flipkart.com' + reviews['href']
        
        stars = [0, 0, 0, 0, 0]
        if all_reviews:
            r = requests.get(all_reviews)
            soup1 = BeautifulSoup(r.text, "html.parser")
            stars_tags = soup1.findAll('div', {'class': '_1uJVNT'})
            for i in range(len(stars_tags)):
                stars[i] = stars_tags[4-i].text.strip() if stars_tags[4-i] else 0
        
        data = {
            'code': [generate_unique_code()],
            'Title': [title],
            'Price': [price],
            'original price': [orgprice],
            'Rating': [ratings],
            'Url2': [all_reviews],
            'Reviews': [reviews_sum],
            'Url': [url],
            'Stars': [stars],
            'Seller': [seller]
        }
        
        sep = {}
        dark = soup.findAll('div', {'class': 'col col-3-12 _2H87wv'})
        light = soup.findAll('div', {'class': 'col col-9-12 _2vZqPX'})
        for i in range(len(light)):
            sep[dark[i].text.strip()] = light[i].text.strip()
        data.update(sep)
        
        df = pd.DataFrame(data)
        df['Total details'] = [sep]
        print(z,url)
        return df
    else:
        print(z,"Failed", url)
        return None

def generate_unique_code():
    characters = string.ascii_letters + string.digits
    unique_code = ''.join(random.sample(characters, 10))
    return unique_code

# Example usage:



In [10]:
ur1='https://www.flipkart.com/sassafras-women-a-line-multicolor-dress/p/itmf850af7a4d4f4?pid=DREGPZGGW7XDY2JH&lid=LSTDREGPZGGW7XDY2JHRJHKC1&marketplace=FLIPKART&q=SASSAFRAS&store=search.flipkart.com&srno=s_1_15&otracker=search&otracker1=search&fm=organic&iid=2714fef2-8430-4b78-97f7-58a2da512cc5.DREGPZGGW7XDY2JH.SEARCH&ppt=None&ppn=None&ssid=4f5vq503ds0000001710020515588&qH=911e6d426b9cde98'
d=scrape_product_data(1,ur1)

1 https://www.flipkart.com/sassafras-women-a-line-multicolor-dress/p/itmf850af7a4d4f4?pid=DREGPZGGW7XDY2JH&lid=LSTDREGPZGGW7XDY2JHRJHKC1&marketplace=FLIPKART&q=SASSAFRAS&store=search.flipkart.com&srno=s_1_15&otracker=search&otracker1=search&fm=organic&iid=2714fef2-8430-4b78-97f7-58a2da512cc5.DREGPZGGW7XDY2JH.SEARCH&ppt=None&ppn=None&ssid=4f5vq503ds0000001710020515588&qH=911e6d426b9cde98


In [11]:
d

,code,Title,Price,original price,Rating,Url2,Reviews,Url,Stars,Seller,...,Ideal For,Type,Style Code,Suitable For,Sleeve Length,Pack of,Neck,Fabric Care,Net Quantity,Total details
0,VcwM3oy0gS,Women A-line Multicolor Dress,₹689,"₹2,299",4.1,https://www.flipkart.com/sassafras-women-a-lin...,90 ratings and 6 reviews,https://www.flipkart.com/sassafras-women-a-lin...,"[9, 3, 9, 16, 53]",SASSAFRAS,...,Women,A-line,SFDRSS10310-Beige,Western Wear,Half Sleeve,1,V-Neck,Dry Clean,1,"{'Color': 'Multicolor', 'Length': 'Above Knee/..."
